# E-Commerce Sales Analysis

Exploratory data analysis of e-commerce transaction data covering revenue performance,
product category trends, geographic distribution, and customer satisfaction metrics.

## Table of Contents

1. [Introduction and Business Objectives](#1-introduction-and-business-objectives)
2. [Data Dictionary](#2-data-dictionary)
3. [Configuration](#3-configuration)
4. [Data Loading](#4-data-loading)
5. [Data Preparation and Transformation](#5-data-preparation-and-transformation)
6. [Revenue Analysis](#6-revenue-analysis)
   - 6.1 Revenue KPIs
   - 6.2 Monthly Revenue Trend
7. [Product Analysis](#7-product-analysis)
8. [Geographic Analysis](#8-geographic-analysis)
9. [Customer Experience Analysis](#9-customer-experience-analysis)
   - 9.1 Review Scores and Delivery Performance
   - 9.2 Order Status Distribution
10. [Summary of Observations](#10-summary-of-observations)

## 1. Introduction and Business Objectives

This notebook analyses e-commerce transaction data to answer the following business questions:

- How did revenue perform in the analysis year compared with the prior year?
- What is the month-over-month revenue growth trend?
- Which product categories drive the most revenue?
- Which US states generate the most sales?
- What is the relationship between delivery speed and customer satisfaction?

The analysis is fully configurable: change `ANALYSIS_YEAR`, `COMPARISON_YEAR`, and
`ANALYSIS_MONTH` in Section 3 to run the same analysis for any period.

## 2. Data Dictionary

### Orders (`orders_dataset.csv`)
| Column | Description |
|--------|-------------|
| order_id | Unique order identifier |
| customer_id | Customer identifier (unique per order) |
| order_status | Fulfillment status: delivered, shipped, canceled, returned, pending, processing |
| order_purchase_timestamp | Timestamp when the order was placed |
| order_approved_at | Timestamp when payment was approved |
| order_delivered_carrier_date | Timestamp when handed to carrier |
| order_delivered_customer_date | Timestamp when delivered to the customer |
| order_estimated_delivery_date | Estimated delivery date |

### Order Items (`order_items_dataset.csv`)
| Column | Description |
|--------|-------------|
| order_id | Unique order identifier |
| order_item_id | Sequence number of the item within the order (1 = first item) |
| product_id | Product identifier |
| seller_id | Seller identifier |
| shipping_limit_date | Deadline by which the seller must ship |
| price | Item price in USD |
| freight_value | Shipping cost in USD |

### Products (`products_dataset.csv`)
| Column | Description |
|--------|-------------|
| product_id | Unique product identifier |
| product_category_name | Category the product belongs to |
| product_weight_g | Product weight in grams |
| product_length_cm, product_height_cm, product_width_cm | Physical dimensions |

### Customers (`customers_dataset.csv`)
| Column | Description |
|--------|-------------|
| customer_id | Order-level identifier (a returning customer has a different customer_id per order) |
| customer_unique_id | Persistent identifier for the same individual |
| customer_state | US state abbreviation (e.g., CA, TX) |

### Reviews (`order_reviews_dataset.csv`)
| Column | Description |
|--------|-------------|
| review_id | Unique review identifier |
| order_id | Order this review belongs to |
| review_score | Rating from 1 (lowest) to 5 (highest) |
| review_comment_title | Short review title |
| review_comment_message | Full review text (may be empty) |

### Derived Metrics
| Term | Definition |
|------|------------|
| Revenue | Sum of item prices (excluding freight) for delivered orders |
| Average Order Value (AOV) | Mean of (sum of item prices per order) |
| Delivery Speed | Days elapsed from order_purchase_timestamp to order_delivered_customer_date |
| MoM Growth | Month-over-month percentage change in monthly revenue |

## 3. Configuration

Edit the values below to change the analysis period. All downstream cells will use these
settings automatically.

- Set `ANALYSIS_MONTH = None` to analyse an entire year.
- Set `ANALYSIS_MONTH` to an integer (1-12) to focus on a single month.

In [ ]:
# ============================================================
# Analysis Configuration
# ============================================================

ANALYSIS_YEAR   = 2023   # Primary year to analyse
COMPARISON_YEAR = 2022   # Year used for year-over-year comparisons
ANALYSIS_MONTH  = None   # None for full-year analysis; set to 1-12 for a single month

DATA_DIR = 'ecommerce_data'

# --- Derived period labels (used in chart titles and print statements) ---
if ANALYSIS_MONTH is None:
    PERIOD_LABEL     = str(ANALYSIS_YEAR)
    COMPARISON_LABEL = str(COMPARISON_YEAR)
else:
    PERIOD_LABEL     = f"{ANALYSIS_YEAR}-{ANALYSIS_MONTH:02d}"
    COMPARISON_LABEL = f"{COMPARISON_YEAR}-{ANALYSIS_MONTH:02d}"

print(f"Analysis period  : {PERIOD_LABEL}")
print(f"Comparison period: {COMPARISON_LABEL}")

## 4. Data Loading

All datasets are loaded through `data_loader.py`. Datetime columns are parsed at load
time to avoid repeated conversions later in the notebook.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import plotly.express as px

from data_loader import (
    load_all_datasets,
    build_sales_data,
    filter_delivered_orders,
    filter_by_period,
    add_delivery_speed,
    categorize_delivery_speed,
)
from business_metrics import (
    calculate_total_revenue,
    calculate_revenue_growth,
    calculate_monthly_revenue,
    calculate_monthly_growth,
    calculate_average_order_value,
    calculate_order_count,
    calculate_category_revenue,
    calculate_state_revenue,
    calculate_review_score_distribution,
    calculate_average_review_score,
    calculate_delivery_review_correlation,
    calculate_average_delivery_time,
    calculate_order_status_distribution,
    summarize_period_metrics,
)

# --- Consistent color palette for all visualizations ---
PRIMARY_COLOR   = '#1f4e79'
SECONDARY_COLOR = '#2e75b6'
CHART_COLORS    = [
    '#1f4e79', '#2e75b6', '#4472c4', '#9dc3e6', '#bdd7ee',
    '#70ad47', '#ffc000', '#ed7d31', '#7030a0', '#00b0f0',
    '#ff4444', '#ab47bc', '#26a69a',
]

In [ ]:
datasets    = load_all_datasets(DATA_DIR)

orders      = datasets['orders']
order_items = datasets['order_items']
products    = datasets['products']
customers   = datasets['customers']
reviews     = datasets['reviews']

print("Datasets loaded successfully.")
print(f"  Orders      : {len(orders):,} records")
print(f"  Order items : {len(order_items):,} records")
print(f"  Products    : {len(products):,} records")
print(f"  Customers   : {len(customers):,} records")
print(f"  Reviews     : {len(reviews):,} records")

## 5. Data Preparation and Transformation

The raw datasets are joined into a single sales table and then filtered to:
1. Keep only **delivered** orders (revenue-generating transactions).
2. Restrict to the configured analysis period and comparison period.
3. Compute derived columns: `delivery_speed_days` and `delivery_time` bucket.

In [ ]:
# Join order items with order metadata
sales_data = build_sales_data(orders, order_items)

# Keep only delivered orders
sales_delivered = filter_delivered_orders(sales_data)

# Filter to the configured periods
sales_analysis    = filter_by_period(sales_delivered, ANALYSIS_YEAR, ANALYSIS_MONTH)
sales_comparison  = filter_by_period(sales_delivered, COMPARISON_YEAR, ANALYSIS_MONTH)

# Add delivery speed
sales_analysis = add_delivery_speed(sales_analysis)
sales_analysis['delivery_time'] = (
    sales_analysis['delivery_speed_days'].apply(categorize_delivery_speed)
)

print(f"Delivered order items ({PERIOD_LABEL})     : {len(sales_analysis):,}")
print(f"Delivered order items ({COMPARISON_LABEL})     : {len(sales_comparison):,}")
print(f"\nOrder statuses in dataset: {sorted(sales_data['order_status'].unique())}")

## 6. Revenue Analysis

This section covers top-level revenue KPIs compared with the prior period and
the month-over-month revenue trend within the analysis year.

### 6.1 Revenue KPIs

In [ ]:
# ---- Revenue ----
revenue_analysis   = calculate_total_revenue(sales_analysis)
revenue_comparison = calculate_total_revenue(sales_comparison)
revenue_growth     = calculate_revenue_growth(revenue_analysis, revenue_comparison)

# ---- Average Order Value ----
aov_analysis   = calculate_average_order_value(sales_analysis)
aov_comparison = calculate_average_order_value(sales_comparison)
aov_growth     = calculate_revenue_growth(aov_analysis, aov_comparison)

# ---- Order Count ----
orders_analysis   = calculate_order_count(sales_analysis)
orders_comparison = calculate_order_count(sales_comparison)
orders_growth     = calculate_revenue_growth(orders_analysis, orders_comparison)

print(f"{'Metric':<30} {COMPARISON_LABEL:>12}  {PERIOD_LABEL:>12}  {'YoY Change':>12}")
print("-" * 70)
print(f"{'Total Revenue (USD)':<30} ${revenue_comparison:>11,.2f}  ${revenue_analysis:>11,.2f}  {revenue_growth*100:>+11.2f}%")
print(f"{'Average Order Value (USD)':<30} ${aov_comparison:>11,.2f}  ${aov_analysis:>11,.2f}  {aov_growth*100:>+11.2f}%")
print(f"{'Total Orders':<30}  {orders_comparison:>11,}   {orders_analysis:>11,}  {orders_growth*100:>+11.2f}%")

### 6.2 Monthly Revenue Trend

The charts below show (a) total revenue per month and (b) month-over-month growth
for the analysis period. Green bars indicate positive MoM growth; red bars indicate
negative growth.

In [ ]:
MONTH_LABELS = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

if ANALYSIS_MONTH is not None:
    print("Monthly trend chart is not applicable for single-month analysis.")
else:
    monthly_rev    = calculate_monthly_revenue(sales_analysis)
    monthly_growth = calculate_monthly_growth(sales_analysis)

    fig, axes = plt.subplots(2, 1, figsize=(12, 9))

    # --- Monthly Revenue ---
    axes[0].bar(
        monthly_rev.index,
        monthly_rev.values,
        color=PRIMARY_COLOR,
        alpha=0.85,
    )
    axes[0].set_title(
        f'Monthly Revenue ({PERIOD_LABEL})',
        fontsize=14, fontweight='bold', pad=12,
    )
    axes[0].set_xlabel('Month')
    axes[0].set_ylabel('Revenue (USD)')
    axes[0].set_xticks(monthly_rev.index)
    axes[0].set_xticklabels([MONTH_LABELS[m - 1] for m in monthly_rev.index])
    axes[0].yaxis.set_major_formatter(
        mticker.FuncFormatter(lambda x, _: f'${x:,.0f}')
    )
    axes[0].grid(axis='y', linestyle='--', alpha=0.4)
    axes[0].spines[['top', 'right']].set_visible(False)

    # --- Month-over-Month Growth ---
    growth_vals = monthly_growth.dropna()
    bar_colors  = ['#70ad47' if v >= 0 else '#c00000' for v in growth_vals.values]
    axes[1].bar(
        growth_vals.index,
        growth_vals.values * 100,
        color=bar_colors,
        alpha=0.85,
    )
    axes[1].axhline(0, color='black', linewidth=0.8)
    axes[1].set_title(
        f'Month-over-Month Revenue Growth ({PERIOD_LABEL})',
        fontsize=14, fontweight='bold', pad=12,
    )
    axes[1].set_xlabel('Month')
    axes[1].set_ylabel('Growth (%)')
    axes[1].set_xticks(growth_vals.index)
    axes[1].set_xticklabels([MONTH_LABELS[m - 1] for m in growth_vals.index])
    axes[1].yaxis.set_major_formatter(
        mticker.FuncFormatter(lambda x, _: f'{x:.1f}%')
    )
    axes[1].grid(axis='y', linestyle='--', alpha=0.4)
    axes[1].spines[['top', 'right']].set_visible(False)

    plt.tight_layout(pad=3.0)
    plt.show()

    avg_mom = monthly_growth.mean()
    print(f"Average monthly growth ({PERIOD_LABEL}): {avg_mom*100:+.2f}%")

## 7. Product Analysis

Revenue breakdown by product category helps identify which lines drive the most
business value and where growth opportunities may exist.

In [ ]:
category_revenue = calculate_category_revenue(sales_analysis, products)

fig, ax = plt.subplots(figsize=(10, 6))

ax.barh(
    category_revenue.index[::-1],
    category_revenue.values[::-1],
    color=SECONDARY_COLOR,
    alpha=0.85,
)
ax.set_title(
    f'Revenue by Product Category ({PERIOD_LABEL})',
    fontsize=14, fontweight='bold', pad=12,
)
ax.set_xlabel('Revenue (USD)')
ax.set_ylabel('Product Category')
ax.xaxis.set_major_formatter(
    mticker.FuncFormatter(lambda x, _: f'${x:,.0f}')
)
ax.grid(axis='x', linestyle='--', alpha=0.4)
ax.spines[['top', 'right']].set_visible(False)

plt.tight_layout()
plt.show()

print(f"\nTop 3 categories by revenue ({PERIOD_LABEL}):")
for rank, (cat, rev) in enumerate(category_revenue.head(3).items(), start=1):
    share = rev / category_revenue.sum() * 100
    print(f"  {rank}. {cat:<30} ${rev:>12,.2f}  ({share:.1f}% of total)")

## 8. Geographic Analysis

The choropleth map shows revenue by US state, highlighting the markets that
contribute most to overall sales in the analysis period.

In [ ]:
state_revenue = calculate_state_revenue(sales_analysis, orders, customers)

fig = px.choropleth(
    state_revenue,
    locations='customer_state',
    color='price',
    locationmode='USA-states',
    scope='usa',
    title=f'Revenue by US State ({PERIOD_LABEL})',
    color_continuous_scale='Blues',
    labels={'price': 'Revenue (USD)', 'customer_state': 'State'},
)
fig.update_layout(
    title_font_size=16,
    coloraxis_colorbar=dict(title='Revenue (USD)'),
)
fig.show()

print(f"\nTop 5 states by revenue ({PERIOD_LABEL}):")
for _, row in state_revenue.head(5).iterrows():
    share = row['price'] / state_revenue['price'].sum() * 100
    print(f"  {row['customer_state']:<6} ${row['price']:>12,.2f}  ({share:.1f}%)")

## 9. Customer Experience Analysis

Customer experience is measured through review scores and delivery performance.
Understanding the relationship between delivery speed and satisfaction can guide
logistics investment decisions.

### 9.1 Review Scores and Delivery Performance

In [ ]:
avg_delivery = calculate_average_delivery_time(sales_analysis)
avg_review   = calculate_average_review_score(sales_analysis, reviews)

print(f"Average delivery time ({PERIOD_LABEL}): {avg_delivery:.1f} days")
print(f"Average review score  ({PERIOD_LABEL}): {avg_review:.2f} / 5.0")

In [ ]:
review_dist     = calculate_review_score_distribution(sales_analysis, reviews)
delivery_review = calculate_delivery_review_correlation(sales_analysis, reviews)

SCORE_COLORS = ['#c00000', '#ff4444', '#ffc000', '#70ad47', '#1a7a1a']
DELIVERY_ORDER = ['1-3 days', '4-7 days', '8+ days']

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# --- Review Score Distribution ---
axes[0].barh(
    review_dist.index,
    review_dist.values,
    color=[SCORE_COLORS[i - 1] for i in review_dist.index],
    alpha=0.85,
)
axes[0].set_title(
    f'Review Score Distribution ({PERIOD_LABEL})',
    fontsize=13, fontweight='bold', pad=12,
)
axes[0].set_xlabel('Proportion of Reviews')
axes[0].set_ylabel('Review Score')
axes[0].xaxis.set_major_formatter(
    mticker.FuncFormatter(lambda x, _: f'{x:.0%}')
)
axes[0].grid(axis='x', linestyle='--', alpha=0.4)
axes[0].spines[['top', 'right']].set_visible(False)

# --- Average Review Score by Delivery Speed ---
dr_ordered = (
    delivery_review
    .set_index('delivery_time')
    .reindex(DELIVERY_ORDER)
    .reset_index()
)
axes[1].bar(
    dr_ordered['delivery_time'],
    dr_ordered['review_score'],
    color=[PRIMARY_COLOR, SECONDARY_COLOR, '#9dc3e6'],
    alpha=0.85,
)
axes[1].set_title(
    f'Average Review Score by Delivery Speed ({PERIOD_LABEL})',
    fontsize=13, fontweight='bold', pad=12,
)
axes[1].set_xlabel('Delivery Time')
axes[1].set_ylabel('Average Review Score')
axes[1].set_ylim(0, 5)
axes[1].grid(axis='y', linestyle='--', alpha=0.4)
axes[1].spines[['top', 'right']].set_visible(False)

plt.tight_layout()
plt.show()

### 9.2 Order Status Distribution

The share of orders in each status provides a high-level view of fulfillment health.

In [ ]:
order_status = calculate_order_status_distribution(orders, ANALYSIS_YEAR)

fig, ax = plt.subplots(figsize=(9, 5))

ax.bar(
    order_status.index,
    order_status.values * 100,
    color=CHART_COLORS[:len(order_status)],
    alpha=0.85,
)
ax.set_title(
    f'Order Status Distribution ({PERIOD_LABEL})',
    fontsize=14, fontweight='bold', pad=12,
)
ax.set_xlabel('Order Status')
ax.set_ylabel('Percentage of Orders (%)')
ax.yaxis.set_major_formatter(
    mticker.FuncFormatter(lambda x, _: f'{x:.1f}%')
)
ax.grid(axis='y', linestyle='--', alpha=0.4)
ax.spines[['top', 'right']].set_visible(False)

plt.tight_layout()
plt.show()

print("\nOrder status breakdown:")
for status, proportion in order_status.items():
    print(f"  {status:<15} {proportion*100:>6.2f}%")

## 10. Summary of Observations

The cell below prints a compact summary of the key metrics for the analysis period.

In [ ]:
summary = summarize_period_metrics(sales_analysis, ANALYSIS_YEAR, ANALYSIS_MONTH)

print("=" * 60)
print(f" Business Metrics Summary - {PERIOD_LABEL}")
print("=" * 60)
print()
print("Revenue Performance")
print(f"  Total Revenue       : ${summary['total_revenue']:>12,.2f}")
print(f"  YoY Change          : {revenue_growth*100:>+12.2f}%")
print()
print("Order Performance")
print(f"  Total Orders        : {summary['order_count']:>13,}")
print(f"  YoY Change          : {orders_growth*100:>+12.2f}%")
print(f"  Avg Order Value     : ${summary['avg_order_value']:>12,.2f}")
print(f"  YoY Change          : {aov_growth*100:>+12.2f}%")
print()
print("Customer Experience")
print(f"  Avg Review Score    : {avg_review:>13.2f} / 5.0")
print(f"  Avg Delivery Time   : {avg_delivery:>12.1f} days")

### Key Observations

The observations below are based on the configured analysis period.
Re-run the notebook with different configuration values to update them.

**Revenue and Orders**
- Total revenue and order count for 2023 were slightly below 2022 levels, indicating
  broadly flat year-over-year performance.
- Average order value remained nearly unchanged between years, suggesting stable
  purchasing behaviour rather than a shift in basket size.
- Month-over-month growth fluctuated between positive and negative, with no sustained
  upward or downward trend through the year.

**Product Categories**
- A small number of categories account for a disproportionate share of revenue.
- The top categories represent a significant concentration; diversifying into
  lower-performing categories could reduce revenue risk.

**Geographic Distribution**
- Revenue is concentrated in a handful of high-population states.
- Lower-revenue states may represent untapped market opportunities.

**Customer Experience**
- The average review score is above 4.0, reflecting generally positive customer
  satisfaction.
- Reviews are skewed toward 5 stars, which makes up approximately half of all ratings.
- Orders delivered in 1-3 days receive the highest average review scores, suggesting
  that faster delivery positively influences satisfaction.
- The overwhelming majority of orders reach "delivered" status, indicating a
  reliable fulfillment process.